Implementation of the Welsh-Powell Algorithm

Sources:

Watson, Gordon. “Comparing the Efficiency of Welsh-Powell Algorithm and Basic Greedy Algorithm for Graph Coloring - Graph Theory.” CopyProgramming, 15 May 2023, copyprogramming.com/howto/why-is-welsh-powell-algorithm-better-than-the-basic-greedy-algorithm-for-graph-coloring.

“Welsh Powell Graph Colouring Algorithm.” GeeksforGeeks, GeeksforGeeks, 29 Oct. 2019, www.geeksforgeeks.org/welsh-powell-graph-colouring-algorithm/.

“Vertex Coloring: The Welsh Powell Algorithm.” Kleemans.Ch, www.kleemans.ch/static/fourcolors/welsh-powell.pdf. Accessed 16 Oct. 2023.

“Graph Coloring Using Greedy Algorithm.” GeeksforGeeks, GeeksforGeeks, 10 Oct. 2023, www.geeksforgeeks.org/graph-coloring-set-2-greedy-algorithm/?ref=ml_lbp\%27.

The Welsh Powell Algorithm is a type of greedy algorithm that is used for determining the fewest number of colors that can be used to color the vertices of a graph with no two adjacent vertices sharing a color. What differentiates this algorithm from a typical greedy algorithm, is that the Welsh Powell Algorithm sorts all of the vertices by decending degree before begining to assign them colors. This approach helps to get the vertices with the highest degree taken care of early on since the higher degree they are, the more likely they are to need a new color. By getting later vertices of lower degree to share a color with these vertices of higher degree, situations where a new color is introduced later in the algorithm can be avoided. 

In [ ]:
sdafjawirjgiojaejiaergoijaeg